# Zbieranie danych ze zdjęć
## Importowanie Bibliotek

In [1]:
import cv2 # Korzystaj z kamerki komputera / obrabianie zdjęć
import mediapipe as mp # Wykorzystaj model do detekcji punktów na twarzy
import json # Pracuj z plikami typu json

## Przygotowywanie programu

In [2]:
mp_face_mesh = mp.solutions.face_mesh # Przygotuj model do ekstrakcji siatki twarzy  
mp_drawing = mp.solutions.drawing_utils # Przygotuj narzędzie do rysowania siatki twarzy na zdjęciu

emotions = {1: "angry", 2: "happy", 3: "neutral", 4: "sad", 5: "surprised"} # Stwórz słownik do prostszego wyszukiwania zdjęć
dummy_variables = {
    "angry":    [True, False, False, False, False],
    "happy":    [False, True, False, False, False],
    "neutral":  [False, False, True, False, False],
    "sad":      [False, False, False, True, False],
    "surprised": [False, False, False, False, True]
} # Stwórz słownik do prostszego oznaczania danych wyjściowych

with open("aa_inputs.json") as inputs_file: # Przy użyciu funkcji 'open()' przy użyciu pliku "imputs.json"... 
    inputs = json.load(inputs_file) # ...zaimportuj dane z tego pliku.

with open("aa_desired_outputs.json") as desired_outputs_file: # Przy użyciu funkcji 'open()' przy użyciu pliku "desired_outputs.json"...
    desired_outputs = json.load(desired_outputs_file) # ...zaimportuj dane z tego pliku.



## Ekstrakcja danych ze zdjęć

In [5]:

def func(integer: int):
    ########################################################
    emotion = emotions[integer] # Sterowanie resztą programu
    ########################################################

    IMAGE_FILES = [] # Definicja bazowej listy nazw zdjęć
    for i in range(70):
        IMAGE_FILES.append(emotion + str(i+1) + ".jpg") # Wypełnienie listy 

    with mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.5) as face_mesh: # Przy użyciu modelu mediapipe do siatki twarzy...
        for idx, file in enumerate(IMAGE_FILES): # Dla każdego ze zdjęć w liście...
            image = cv2.imread(file) # Przy użyciu biblioteki cv2 odczytaj zdjęcie pod zmienną 'file'.
            results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)) # Konwersja bitmapy BGR (używanej przez cv2) do RGB (używanej przez mediapipe).
        
            face_coordinates = [] # Definicja bazowej listy koordynatów punktów na twarzy
            
            if not results.multi_face_landmarks: # W przypadku braku wykrycia twarzy program przejdzie do następnego zdjęcia.
                continue
            
            for face in results.multi_face_landmarks: # Szczerze nie wiem po co jest ta pętla, ale na stack overflow pisali, żeby tak zrobić bez podania przyczyny (obstawiam, że to jest w razie wykrycia większej ilości twarzy, ale nie jestem pewny).
                for landmark in face.landmark: # Dla każdgo z koordynatów siatki twarzy dopisz do listy 'face_coordinates' wartość tego koordynatu.
                    face_coordinates.append(landmark.x)
                    face_coordinates.append(landmark.y)
                    face_coordinates.append(landmark.z)
            
            inputs["Values"].append(face_coordinates) # Do obiektu typu json 'wejścia' dopisz koordynaty twarzy.
            desired_outputs["Values"].append(dummy_variables[emotion]) # Do obiektu typu json 'oczekiwane wyjścia' dopisz wartości oczekiwanych emocji.


for i in range(5):
    func(i+1)

In [6]:
with open("aa_desired_outputs.json", 'w') as desired_outputs_file: # Przy użyciu funkcji 'open()' przy użyciu pliku "imputs.json"... 
    json.dump(desired_outputs, desired_outputs_file) # ...załaduj dane do tego pliku.

with open("aa_inputs.json", 'w') as inputs_file: # Przy użyciu funkcji 'open()' przy użyciu pliku "imputs.json"... 
    json.dump(inputs, inputs_file) # ...załaduj dane do tego pliku.